In [1]:
from unsloth import FastLanguageModel
import torch, psutil, os, gc
from datasets import load_dataset, concatenate_datasets
from trl import SFTTrainer, SFTConfig
from datetime import datetime
import requests
from MyModule import get_current_time, get_current_date, convert_to_grid, get_weather, search_address

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [2]:
model_name = "../model/kanana-1.5-2.1b-instruct-2505"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length = 1024,
    dtype = None,
    load_in_4bit = True
)

==((====))==  Unsloth 2026.1.2: Fast Llama patching. Transformers: 4.57.3.
   \\   /|    NVIDIA GeForce RTX 3060. Num GPUs = 1. Max memory: 12.0 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu128. CUDA: 8.6. CUDA Toolkit: 12.8. Triton: 3.5.1
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 8,
    target_modules = [
        "q_proj", "k_proj", "v_proj",
        "o_proj",
        "gate_proj", "up_proj", "down_proj"
    ],
    lora_alpha = 16,
    lora_dropout = 0.05,
    bias = "none",
    use_gradient_checkpointing = True
)

Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.05.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth 2026.1.2 patched 32 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


In [4]:
dataset1 = load_dataset("json", data_files="./data/train_data_singleturn.jsonl")
dataset2 = load_dataset("json", data_files="./data/train_data_function_call_kanana.jsonl")

dataset = concatenate_datasets([
    dataset1['train'],
    dataset2['train']
]).shuffle(seed=42)
dataset[0]

Generating train split: 0 examples [00:00, ? examples/s]

{'messages': [{'role': 'user', 'content': '지금 몇 시지?'},
  {'role': 'assistant',
   'content': '<function=get_current_time>{}\n\n</function>'},
  {'role': 'tool', 'content': '{"hour":13,"minute":20,"second":40}'},
  {'role': 'assistant', 'content': '현재 시각은 13시 20분입니다.'}]}

In [5]:
# 함수 매핑
TOOLS = {
    "get_current_time": get_current_time,
    "get_current_date": get_current_date,
    "get_weather": get_weather,
    "search_address": search_address,
}

# 2️⃣ Tools 정의 (Kanana에 전달)
TOOL_DEFINITIONS = [
    {
        "name": "get_current_time",
        "description": "현재 시간을 반환합니다",
        "parameters": {
            "type": "object",
            "properties": {},
            "required": []
        }
    },
    {
        "name": "get_current_date",
        "description": "현재 날짜를 반환합니다",
        "parameters": {
            "type": "object",
            "properties": {},
            "required": []
        }
    },
    {
        "name": "get_weather",
        "description": "특정 지역의 날씨 정보를 조회합니다",
        "parameters": {
            "type": "object",
            "properties": {
                "city": {
                    "type": "string",
                    "description": "지역 이름"
                }
            },
            "required": ["city"]
        }
    },
    {
        "name": "search_address",
        "description": "주소를 검색합니다",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "검색어"
                }
            },
            "required": ["location"]
        }
    }
]

In [6]:
# assistant_only_loss = False일 경우
def formatting_prompts_func(examples):
    
    texts = []

    for message in examples['messages']:
        text = tokenizer.apply_chat_template(
            message,
            tokenize=False,
            add_generation_prompt=False,
            tools=TOOL_DEFINITIONS
        )

        texts.append(text)

    return {'text': texts}

formatted_dataset = dataset.map(formatting_prompts_func, batched=True, remove_columns=dataset.column_names)
print(formatted_dataset[0])

split_dataset = formatted_dataset.train_test_split(test_size=0.1)
train_ds = split_dataset["train"]
eval_ds = split_dataset["test"]

Map:   0%|          | 0/5056 [00:00<?, ? examples/s]

{'text': '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\n\n\nYou have access to the following functions:\n\nUse the function \'get_current_time\' to \'현재 시간을 반환합니다\'\n{"name": "get_current_time", "description": "현재 시간을 반환합니다", "parameters": {"type": "object", "properties": {}, "required": []}}\n\nUse the function \'get_current_date\' to \'현재 날짜를 반환합니다\'\n{"name": "get_current_date", "description": "현재 날짜를 반환합니다", "parameters": {"type": "object", "properties": {}, "required": []}}\n\nUse the function \'get_weather\' to \'특정 지역의 날씨 정보를 조회합니다\'\n{"name": "get_weather", "description": "특정 지역의 날씨 정보를 조회합니다", "parameters": {"type": "object", "properties": {"city": {"type": "string", "description": "지역 이름"}}, "required": ["city"]}}\n\nUse the function \'search_address\' to \'주소를 검색합니다\'\n{"name": "search_address", "description": "주소를 검색합니다", "parameters": {"type": "object", "properties": {"location": {"type": "string", "description": "검색어"}}, "required": ["location"]}}\n\n\nTh

In [7]:
# # assistant_only_loss = True일 경우
# def formatting_prompts_func(examples):
#     if len(examples["messages"]) > 0 and isinstance(examples["messages"][0], list):
#         messages_list = examples["messages"] # 예) {"messages": [ [채팅1], [채팅2], [채팅3] ]}
#     else:
#         # 단일 샘플 처리 (SFTTrainer 검증용) 
#         messages_list = [examples["messages"]] # 예) {"messages": [채팅1]}

#     texts = []
#     for message in messages_list:
#         text = tokenizer.apply_chat_template(
#             message,
#             tokenize=False,
#             add_generation_prompt=False,
#             tools=TOOL_DEFINITIONS
#         )
#         texts.append(text)

#     # SFTTrainer의 formatting_func는 반드시 '문장들의 리스트'를 반환해야 한다.
#     return texts

# split_dataset2 = dataset.train_test_split(test_size=0.1)
# train_ds2 = split_dataset2["train"]
# eval_ds2 = split_dataset2["test"]

In [8]:
args = SFTConfig(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 2,
        warmup_steps = 5,
        # max_steps = 400,
        num_train_epochs=1,
        learning_rate = 5e-5, 
        bf16 = True,
        seed = 1234,
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        output_dir = "outputs",

        logging_steps=100,
        eval_strategy="steps",
        eval_steps=100,
        save_strategy="steps",
        save_steps=100,
        save_total_limit=3,
        load_best_model_at_end=True,
        metric_for_best_model="loss",
        greater_is_better=False,

        # SFTConfig의 파라미터
        packing=False,
        dataset_text_field="text",
        # model_init_kwargs={"trust_remote_code": True},
        # assistant_only_loss=True,
)

trainer = SFTTrainer(
    model=model,
    train_dataset=train_ds,
    eval_dataset=eval_ds,
    processing_class=tokenizer,
    # formatting_func=formatting_prompts_func,
    args=args
)

Unsloth: Tokenizing ["text"] (num_proc=4):   0%|          | 0/4550 [00:00<?, ? examples/s]

Unsloth: Tokenizing ["text"] (num_proc=4):   0%|          | 0/506 [00:00<?, ? examples/s]

🦥 Unsloth: Padding-free auto-enabled, enabling faster training.


In [9]:
trainer.train()

The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 4,550 | Num Epochs = 1 | Total steps = 1,138
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 2
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 2 x 1) = 4
 "-____-"     Trainable parameters = 11,501,568 of 2,098,486,272 (0.55% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss,Validation Loss
100,0.738000,0.275219
200,0.260800,0.249825
300,0.245700,0.236403
400,0.235100,0.228387
500,0.230400,0.224229
600,0.224500,0.220921
700,0.219200,0.218297
800,0.219900,0.216113
900,0.217200,0.215195
1000,0.214800,0.214087


Unsloth: Not an error, but LlamaForCausalLM does not accept `num_items_in_batch`.
Using gradient accumulation will be very slightly less accurate.
Read more on gradient accumulation issues here: https://unsloth.ai/blog/gradient


TrainOutput(global_step=1138, training_loss=0.2722814535633751, metrics={'train_runtime': 3214.0768, 'train_samples_per_second': 1.416, 'train_steps_per_second': 0.354, 'total_flos': 2.5308356619446784e+16, 'train_loss': 0.2722814535633751, 'epoch': 1.0})

### 테스트

In [1]:
import torch, psutil, os, gc, re, json
from datetime import datetime, timedelta
from unsloth import FastLanguageModel
from transformers import GenerationConfig, TextIteratorStreamer
from peft import PeftModel
from threading import Thread
from memory_monitor import MemoryMonitor
import requests
from MyModule import get_current_time, get_current_date, get_weather, search_address

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [2]:
monitor = MemoryMonitor(device_index=0)
torch.cuda.empty_cache()
gc.collect()

monitor.print_usage(label="Before Loading")

# 모델 로드
model_name = "../model/kanana-1.5-2.1b-instruct-2505"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length = 1024,
    dtype = None,
    load_in_4bit = True
)

model = PeftModel.from_pretrained(model, "./outputs/checkpoint-1100")

# 추론 모드로 전환
FastLanguageModel.for_inference(model)

monitor.print_usage(label="After Loading")

# 1 token 생성
prompt = "안녕하세요."
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
_ = model.generate(**inputs, max_new_tokens=1)
monitor.print_usage(label="After Generating")

[Before Loading] RAM: 993.74 MB | VRAM: 1185.86 MB
==((====))==  Unsloth 2026.1.2: Fast Llama patching. Transformers: 4.57.3.
   \\   /|    NVIDIA GeForce RTX 3060. Num GPUs = 1. Max memory: 12.0 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu128. CUDA: 8.6. CUDA Toolkit: 12.8. Triton: 3.5.1
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
[After Loading] RAM: 1353.05 MB | VRAM: 3302.61 MB
[After Generating] RAM: 1715.68 MB | VRAM: 3357.80 MB


In [3]:
# 함수 매핑
TOOLS = {
    "get_current_time": get_current_time,
    "get_current_date": get_current_date,
    "get_weather": get_weather,
    "search_address": search_address,
}

# 2️⃣ Tools 정의 (Kanana에 전달)
TOOL_DEFINITIONS = [
    {
        "name": "get_current_time",
        "description": "현재 시간을 반환합니다",
        "parameters": {
            "type": "object",
            "properties": {},
            "required": []
        }
    },
    {
        "name": "get_current_date",
        "description": "현재 날짜를 반환합니다",
        "parameters": {
            "type": "object",
            "properties": {},
            "required": []
        }
    },
    {
        "name": "get_weather",
        "description": "특정 지역의 날씨 정보를 조회합니다",
        "parameters": {
            "type": "object",
            "properties": {
                "city": {
                    "type": "string",
                    "description": "지역 이름"
                }
            },
            "required": ["city"]
        }
    },
    {
        "name": "search_address",
        "description": "주소를 검색합니다",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "검색어"
                }
            },
            "required": ["location"]
        }
    }
]

In [4]:
generation_config = GenerationConfig(
    max_new_tokens=512,  # 생성할 최대 토큰 수
    temperature=0.6,
    top_p=0.95,
    top_k=50,
    do_sample=False,
    eos_token_id=tokenizer.eos_token_id,  # 끝 토큰 명시
    pad_token_id=tokenizer.pad_token_id,
)

def stream_with_langchain(question):
    messages = [
        {"role": "user", "content": question}
    ]
    
    formatted_text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
        tools=TOOL_DEFINITIONS
    )

    inputs = tokenizer(
        formatted_text,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=1024,
    )

    input_ids = inputs["input_ids"].to(model.device)
    attention_mask = inputs["attention_mask"].to(model.device)
    
    streamer = TextIteratorStreamer(
        tokenizer,
        skip_special_tokens=True,   # <|end|> 같은 특수 토큰 건너뛰기
        skip_prompt=True            # 입력 프롬프트 건너뛰기 (응답만 보기)
    )           
    
    # Thread가 있어야 메인 스레드에서 백그라운드로 generate()가 실행되고
    # 서브 스레드에서 streamer로 토큰을 실시간으로 읽을 수 있음
    # 따라서 동시에 병렬로 처리 가능

    thread = Thread(
        target=model.generate, 
        kwargs=dict(
            input_ids=input_ids,
            attention_mask=attention_mask,
            generation_config=generation_config,
            streamer=streamer,
        )
    )
    thread.start()
    
    full_response = ""
    for text in streamer:
        print(text, end="", flush=True)
        full_response += text
    
    thread.join()
    print()     # 줄바꿈

    pattern = r"<function=(\w+)>(.+?)</function>"
    match = re.search(pattern, full_response)

    if match:
        # 함수 호출 형태 감지됨
        func_name = match.group(1)  # get_weather
        args_str = match.group(2) # "location": "Seoul"

        args = json.loads(args_str)

        tool_result = TOOLS[func_name](**args)

        messages.append({
            "role": "assistant",
            "content": full_response
        })
        messages.append({
            "role": "tool",
            "content": json.dumps(tool_result, ensure_ascii=False)
        })

        formatted_text_2 = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True,
        )

        inputs2 = tokenizer(
            formatted_text_2,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=1024,
        )

        streamer2 = TextIteratorStreamer(
            tokenizer,
            skip_special_tokens=True,   # <|end|> 같은 특수 토큰 건너뛰기
            skip_prompt=True            # 입력 프롬프트 건너뛰기 (응답만 보기)
        )

        thread2 = Thread(
            target=model.generate, 
            kwargs=dict(
                input_ids=inputs2["input_ids"].to(model.device),
                attention_mask=inputs2["attention_mask"].to(model.device),
                generation_config=generation_config,
                streamer=streamer2,
            )
        )
        thread2.start()
        
        final_response = ""
        for text in streamer2:
            print(text, end="", flush=True)
            final_response += text
        
        thread2.join()
        print()

        return final_response

    return full_response

In [8]:
question = "대구백화점 주소 알려줘"
print(f"Q: {question}")
print("A: ", end="", flush=True)
response = stream_with_langchain(question)

Q: 대구백화점 주소 알려줘
A: <function=search_address>{"location":"대구백화점"}</function>
{'road_address_name': '대구 중구 명덕로 333', 'place_name': '대구백화점 대백프라자점'}
대구백화점의 주소는 대구 중구 명덕로 333에 위치한 '대구백화점 대백프라자점'이 있습니다.


In [14]:
question = "서변동 주소 알려줘."
print(f"Q: {question}")
print("A: ", end="", flush=True)
response = stream_with_langchain(question)

Q: 서변동 주소 알려줘.
A: <function=search_address>{"location":"서변동"}</function>
{'road_address_name': '대구 북구 호국로 219', 'place_name': '서변온천'}
서변동의 주소는 대구 북구 호국로 219에 위치한 '서변온천'이 있습니다.


### GGUF 변환

In [16]:
from unsloth import FastLanguageModel

# 모델 및 아답터 로드 (한 번에 불러오기)
adapter_path = "./outputs/checkpoint-1100" # 학습된 아답터

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = adapter_path, # 아답터 경로를 넣으면 베이스 모델까지 자동으로 찾아서 합친다.
    max_seq_length = 1024,
    dtype = None,
    load_in_4bit = True,
)

# 젯슨용 GGUF로 저장하기
model.save_pretrained_merged(
    "kanana_merged_temp",    # 저장될 폴더명/파일명
    tokenizer,
    save_method = "merged_16bit"
)

==((====))==  Unsloth 2026.1.2: Fast Llama patching. Transformers: 4.57.3.
   \\   /|    NVIDIA GeForce RTX 3060. Num GPUs = 1. Max memory: 12.0 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu128. CUDA: 8.6. CUDA Toolkit: 12.8. Triton: 3.5.1
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth 2026.1.2 patched 32 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


Detected local model directory: /home/user/work/model/kanana-1.5-2.1b-instruct-2505
Found HuggingFace hub cache directory: /home/user/.cache/huggingface/hub


Unsloth: Preparing safetensor model files: 100%|██████████| 1/1 [00:33<00:00, 33.96s/it]


Copied model.safetensors from local model directory


Unsloth: Merging weights into 16bit: 100%|██████████| 1/1 [01:46<00:00, 106.48s/it]


Unsloth: Merge process complete. Saved to `/home/user/work/LLM/kanana_merged_temp`
